In [2]:
from typing import Literal
from collections import deque

# 数式の木

## Node クラス

数式の要素を表すノード

### メソッド

- value: str<br/>
  数式の 1 要素

- left: Node | None<br/>
  演算子の左辺

- right: Node | None<br/>
  演算子の右辺


In [3]:
class Node:
    def __init__(self, value: str, left=None, right=None) -> None:
        self.value: str = value
        self.left: Node | None = left
        self.right: Node | None = right

    def __str__(self) -> str:
        return self.value

## 関数

### ノードにする

引数`s: Node | str`を受け取って、`s`が`isinstance(s, Node) >>> True`ならそのまま返し、`False`なら`value`を`s`とする`Node`を返す


In [4]:
def to_node(s: Node | str) -> Node:
    if isinstance(s, Node):
        return s
    return Node(s)

## 関数

いずれも数式`formula: str`を受け取り数式の木(Node)にして根のノードを返す

### 前置記法(post 関数)

後置記法の数式を引数にとり、数式の木にしてその根のノードを返す関数

### 後置記法(pre 関数)

前置記法の数式を引数にとり、数式の木にしてその根のノードを返す関数

### 中置記法(infix 関数)

中置記法の数式を引数にとり、数式の木にしてその根のノードを返す関数


In [5]:
def post(formula: str) -> Node:
    formula_deq = deque(formula.split())
    stack = []
    while len(formula_deq) > 0:
        s = formula_deq.popleft()
        if s in ["+", "-", "*", "/"]:
            right = stack.pop()
            left = stack.pop()
            new_node = Node(s, to_node(left), to_node(right))
            stack.append(new_node)
        else:
            stack.append(s)
    return stack[0]


def pre(formula: str) -> Node:
    formula_list = formula.split()
    formula_list.reverse()
    stack = []
    formula_deq = deque(formula_list)
    while len(formula_deq) > 0:
        s = formula_deq.popleft()
        if s in ["+", "-", "*", "/"]:
            left = stack.pop()
            right = stack.pop()
            new_node = Node(s, to_node(left), to_node(right))
            stack.append(new_node)
        else:
            stack.append(s)
    return stack[0]


def infix(formula: str) -> Node:
    order = {"+": 2, "-": 2, "*": 1, "/": 1, "(": 3}
    operator = []
    stack = []
    for s in formula.replace("(", " ( ").replace(")", " ) ").split():
        if s == ")":
            while True:
                t = operator.pop()
                if t == "(":
                    break
                stack.append(t)
        elif s == "(":
            operator.append(s)
        elif s in ["+", "-", "*", "/"]:
            while len(operator) > 0:
                t = operator.pop()
                if order[t] > order[s]:
                    operator.append(t)
                    break
                stack.append(t)
                if order[t] == order[s]:
                    break
            operator.append(s)
        else:
            stack.append(s)
    while len(operator) > 0:
        stack.append(operator.pop())
    return post(" ".join(stack))

## 関数

### make_formula_tree 関数

各記法の数式をそれぞれに合った、数式の木にして返す関数を呼び出す関数

#### 引数

- formula: str<br/>
  数式
- type: Literal["pre", "in", "post"]

  - 数式の記法
  - pre->前置記法
  - in->中置記法
  - post->後置記法

#### 返り値

数式の木の根のノード


In [6]:
def make_formula_tree(formula: str, type: Literal["pre", "in", "post"]) -> Node:
    if type == "post":
        return post(formula)
    if type == "pre":
        return pre(formula)
    return infix(formula)

## 関数

いずれも数式の木の根`node: Node`を受け取り任意の記法の数式にして返す

### 前置記法(return_postorder 関数)

数式の根のノードを引数にとり、後置記法の数式を返す関数

### 中置記法(return_inorder 関数)

数式の根のノードを引数にとり、中置記法の数式を返す関数

### 後置記法(return_preorder 関数)

数式の根のノードを引数にとり、前置記法の数式を返す関数


In [7]:
def return_preorder(node: Node):
    right = ""
    left = ""
    if node.left is not None:
        left = return_preorder(node.left)
    if node.right is not None:
        right = return_preorder(node.right)
    return f"{node.value} {left} {right}"


def return_inorder(node: Node):
    right = ""
    left = ""
    if node.value == "*":
        if node.left is not None:
            if node.left.value in ["+", "-"]:
                left = f"( {return_inorder(node.left)} ) "
            else:
                left = return_inorder(node.left)
        if node.right is not None:
            if node.right.value in ["+", "-"]:
                right = f"( {return_inorder(node.right)} ) "
            else:
                right = return_inorder(node.right)
    elif node.value == "/":
        if node.left is not None:
            if node.left.value in ["+", "-", "*", "/"]:
                left = f"( {return_inorder(node.left)} ) "
            else:
                left = return_inorder(node.left)
        if node.right is not None:
            if node.right.value in ["+", "-", "*", "/"]:
                right = f"( {return_inorder(node.right)} ) "
            else:
                right = return_inorder(node.right)
    else:
        if node.left is not None:
            left = return_inorder(node.left)
        if node.right is not None:
            right = return_inorder(node.right)
    return f"{left} {node.value} {right}"


def return_postorder(node: Node):
    right = ""
    left = ""
    if node.left is not None:
        left = return_postorder(node.left)
    if node.right is not None:
        right = return_postorder(node.right)
    return f"{left} {right} {node.value}"

## 関数

### change_formula_type 関数

各記法の数式を任意の記法の数式に変換する関数

#### 引数

- formula: str<br/>
  数式

- expected_type: Literal["pre", "in", "post"]<br/>
  変換後の数式の記法
  - pre->前置記法
  - in->中置記法
  - post->後置記法

#### 返り値

変換後の数式


In [8]:
def change_formula_type(
    formula: str, expected_type: Literal["pre", "in", "post"]
) -> str:
    type: Literal["pre", "in", "post"]
    if formula[0] in ["+", "-", "*", "/"]:
        type = "pre"
    elif formula[len(formula) - 1] in ["+", "-", "*", "/"]:
        type = "post"
    else:
        type = "in"
    formula_tree = make_formula_tree(formula, type)
    result: str = {
        "pre": return_preorder,
        "in": return_inorder,
        "post": return_postorder,
    }[expected_type](formula_tree)
    return " ".join(result.split())

In [9]:
print(change_formula_type("10 3 4 + 5 2 - * +", "in"))
print(change_formula_type("10 3 4 + 5 2 - * +", "pre"))
print(change_formula_type("10 + ( 3 + 4 ) * ( 5 - 2 )", "pre"))
print(change_formula_type("10 + ( 3 + 4 ) * ( 5 - 2 )", "post"))

10 + ( 3 + 4 ) * ( 5 - 2 )
+ 10 * + 3 4 - 5 2
+ 10 * + 3 4 - 5 2
10 3 4 + 5 2 - * +


In [10]:
print(change_formula_type("(7 - 2) * (3 + 2) + 2 * 3", "post"))
print(change_formula_type("+ * 3 4 * + 1 2 - 8 6", "in"))

7 2 - 3 2 + * 2 3 * +
3 * 4 + ( 1 + 2 ) * ( 8 - 6 )
